<a href="https://colab.research.google.com/github/dkimds/goormthon/blob/main/%08clf_naver_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 뉴스 카테고리 분류기 만들기

## 자연어처리 라이브러리 설치

In [ ]:
# nltk.download()  # 텍스트 데이터 처리를 위한 패키지 다운로더

# Download following packages
# Corpora : stopwords, wordnet
# Models : averaged_perceptron_tagger, d, punkt
import nltk
nltk.download()

In [ ]:
# Install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

## 데이터셋 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 원하는 세부 경로로 변경이 불가능합니다.

clf_news_MultinomialNB_v1.pkl  result_231210_0127.csv  result_231210_0221.csv  sample_data
result_231210_0111.csv	       result_231210_0206.csv  result_231210_0239.csv


In [ ]:
import pandas as pd
import numpy as np

csv_list = ['/content/drive/MyDrive/data/result_231210_0111.csv',
            '/content/drive/MyDrive/data/result_231210_0206.csv',
            '/content/drive/MyDrive/data/result_231210_0239.csv',
            '/content/drive/MyDrive/data/result_231210_0127.csv',
            '/content/drive/MyDrive/data/result_231210_0221.csv']
news_df = pd.DataFrame()
for csv in csv_list:
    df_data = pd.read_csv(csv)
    news_df = news_df.append(df_data)

print('전체 리뷰 수:', len(news_df))
news_df.head()

전체 리뷰 수: 2316


<ipython-input-4-aee8af246a20>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-4-aee8af246a20>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-4-aee8af246a20>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-4-aee8af246a20>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-4-aee8af246a20>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news

,date,category,press,title,document,link
0,2023.11.29. 오후 12:00,사회,연합뉴스,3분기 출산율 0.7명 '역대 최저'…0.6명대 추락 '빨간불',"통계청, 9월 인구동향…작년 3분기比 0.1명↓47개월째 인구 자연감소(세종=연합뉴...",https://n.news.naver.com/mnews/article/001/001...
1,2023.11.29. 오후 6:04,경제,한국경제,통계나올 때마다 '쇼크'…3분기 출산율 0.7명 또 역대 최저,통계청 인구동향17개 광역시·도 일제히 하락광주 0.2명 줄어 하락폭 최대 인구 4...,https://n.news.naver.com/mnews/article/015/000...
2,2023.11.30. 오전 5:03,사회,서울신문,3분기도 출산율 최저… 올 0.7명도 위태롭다,1년 전보다 0.1명 줄어들어 0.7명9월 출생아 수·혼인도 가장 낮아다문화 혼인은...,https://n.news.naver.com/mnews/article/081/000...
3,2023.11.29. 오후 3:29,경제,한경비즈니스,“이러다 한국 진짜 망한다”...또 다시 ‘출산율 쇼크’,3분기 합계출산율 0.7명 기록3분기 기준 역대 최저치4분기에는 더 떨어질 가능성 ...,https://n.news.naver.com/mnews/article/050/000...
4,2023.11.29. 오후 9:33,경제,TV조선,"3분기 합계 출산율, 0.7명…올해 출생아 20만 명대 턱걸이","[앵커]출산율 감소, 혼인율 감소 저도 계속 전하는 입장에서 뾰족한 대책이 없...",https://n.news.naver.com/mnews/article/448/000...


In [ ]:
def tokenize_and_stemming(sentence, tokenizer):

    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence)

    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
        # pos 기준 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
        # 리뷰 텍스트 상 형태소의 순서를 고려하는 경우 의미를 갖게 됨 : "VerbPrefix" (잘/안/못), "Conjunction"(하지만/그러나/그리고), "KoreanParticle"(ㅋㅋㅋ/ㅎㅎㅎ/ㅠㅠ)
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)

    return sentence_tokenized

In [ ]:
from konlpy.tag import Okt
import tqdm
import re

okt = Okt()
tokenized_sentences = []

for sentence in tqdm.tqdm(news_df['document']):
    try:
        tokenized_sentences.append(tokenize_and_stemming(sentence, okt))
    except:
        print("Error occured at :", sentence)
        tokenized_sentences.append([])

news_df['document'] = tokenized_sentences

 41%|████      | 942/2316 [01:56<01:02, 21.89it/s]

Error occured at : nan


 44%|████▍     | 1015/2316 [02:00<01:05, 19.90it/s]

Error occured at : nan


 46%|████▌     | 1055/2316 [02:02<00:41, 30.29it/s]

...

 56%|█████▋    | 1305/2316 [02:13<00:43, 23.24it/s]

Error occured at : nan


 57%|█████▋    | 1319/2316 [02:13<00:32, 30.35it/s]

Error occured at : nan


100%|██████████| 2316/2316 [03:51<00:00, 10.00it/s]


In [ ]:
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
categories = news_df['category'].tolist()
le.fit(categories)
news_df['category'] = le.transform(categories)

In [ ]:
news_df.head()

,date,category,press,title,document,link
0,2023.11.29. 오후 12:00,2,연합뉴스,3분기 출산율 0.7명 '역대 최저'…0.6명대 추락 '빨간불',"[통계청, 인구, 동향, 작년, 개월, 인구, 자연, 감소, 세종, 연합뉴스, 이준...",https://n.news.naver.com/mnews/article/001/001...
1,2023.11.29. 오후 6:04,1,한국경제,통계나올 때마다 '쇼크'…3분기 출산율 0.7명 또 역대 최저,"[통계청, 인구, 동향, 광역시, 일제, 하락, 광주, 줄다, 하락, 최대, 인구,...",https://n.news.naver.com/mnews/article/015/000...
2,2023.11.30. 오전 5:03,2,서울신문,3분기도 출산율 최저… 올 0.7명도 위태롭다,"[전보, 줄어들다, 출생, 혼인, 가장, 낮다, 다문화, 혼인, 늘다, 올해, 합계...",https://n.news.naver.com/mnews/article/081/000...
3,2023.11.29. 오후 3:29,1,한경비즈니스,“이러다 한국 진짜 망한다”...또 다시 ‘출산율 쇼크’,"[합계출산율, 기록, 기준, 역대, 최저, 떨어지다, 가능성, 높다, 통계청, 발표...",https://n.news.naver.com/mnews/article/050/000...
4,2023.11.29. 오후 9:33,1,TV조선,"3분기 합계 출산율, 0.7명…올해 출생아 20만 명대 턱걸이","[앵커, 출산율, 감소, 혼인, 감소, 계속, 전하, 입장, 뾰족, 대책, 없다, ...",https://n.news.naver.com/mnews/article/448/000...


## 트레이닝/테스트 셋으로 나누기

In [ ]:
from sklearn import model_selection

train_x, test_x, train_y, test_y = model_selection.train_test_split(news_df['document'], news_df['category'],
                                                                    test_size=0.3,
                                                                    random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # CountVectorizer

def list_to_list(doc):
    return doc

vectorizer = TfidfVectorizer(tokenizer=list_to_list, token_pattern=None, lowercase=False) # CountVectorizer 적용 시 아주 약간 성능 저하 (88.15% -> 88.02%)
vectorizer.fit(train_x)

train_x_vector = vectorizer.transform(train_x) # .toarray() 함수를 연달아 적용하면 np.array 형태로 만들어진 행렬을 확인할 수 있으나, RAM이 초과되어 Colab이 초기화됩니다.
test_x_vector = vectorizer.transform(test_x)

## 모델 생성

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(force_alpha=True)

## 모델 학습

In [ ]:
import numpy as np

x = train_x_vector.toarray()

clf.fit(train_x_vector, train_y)

MultinomialNB(force_alpha=True)

## 테스트 데이터 예측

In [ ]:
pred_y = clf.predict(test_x_vector)

# 모델 평가

## Accuracy score

In [ ]:
clf.score(test_x_vector, test_y)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.6561151079136691

## Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       1.00      0.05      0.10        58
           1       0.60      0.91      0.73       220
           2       0.72      0.73      0.72       216
           3       0.68      0.79      0.73        97
           4       0.68      0.35      0.46        54
           5       0.00      0.00      0.00        25
           6       0.00      0.00      0.00        25

    accuracy                           0.66       695
   macro avg       0.53      0.40      0.39       695
weighted avg       0.64      0.66      0.60       695



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_y, pred_y))

[[  3  43   5   0   7   0   0]
 [  0 200  14   4   2   0   0]
 [  0  26 157  33   0   0   0]
 [  0   6  14  77   0   0   0]
 [  0  33   2   0  19   0   0]
 [  0  21   4   0   0   0   0]
 [  0   2  23   0   0   0   0]]


## 모델 저장

In [ ]:
import joblib

joblib.dump(clf, 'clf_news_MultinomialNB_v1.pkl', compress=True)

['clf_news_MultinomialNB_v1.pkl']